### Un script Python pour réparer les champs vides dans les fichier IATI XML..
<br>
![](http://www.aidtransparency.net/annualreport2015/images/furniture/logo.png)
<br>
#### Problème
En naviguant sur la section "Data Quality" du [dashboard AFD](http://dashboard.iatistandard.org/publisher/afd.html) de l'IATI, on peut voir que les fichiers xml de l'AFD sont corrompus. Par exemple si l'on regarde de plus près le fichier XML Afghanistan, [fr-AFD-AF.xml](https://www.data.gouv.fr/s/resources/donnees-de-laide-au-developpement-de-lafd-1/20161216-144706/fr-AFD-AF.xml), en cliquant sur l'outil ["validator"](http://validator.iatistandard.org/?url=https://www.data.gouv.fr/s/resources/donnees-de-laide-au-developpement-de-lafd-1/20161216-144706/fr-AFD-AF.xml) puis "Test Validation" et en explorant l'onglet "Extra info" on obtient pour le fichier XML [Afghanistan](https://www.data.gouv.fr/s/resources/donnees-de-laide-au-developpement-de-lafd-1/20161216-144706/fr-AFD-AF.xml)
les erreurs suivantes:

    94	Error 1824	Element 'activity-date', attribute 'iso-date': '' is not a valid value of the atomic type 'xs:date'.
    133	Error 1824	Element 'activity-date', attribute 'iso-date': '' is not a valid value of the atomic type 'xs:date'.
    ...

Ces erreurs indiquent qu'auncune date n'a été remplie là où une date était obligatoire, par exemple:

    <activity-date type="end-planned" iso-date=""/> 
 
#### Réparation
Dans le cas où il n'existe pas d'information sur la date de fin planifiée du projet, le standard impose que la ligne soit supprimer.

#### Protocole
Ce script à pour objectif de réparer les fichiers corrompus. Pour ce faire il va:
1. Scrapper la page AFD du portail IATI
2. Récupérer les liens vers les fichiers xml (et proposer un lien vers la page validator du fichier correspondant)
3. Télécharger les fichier originaux corrompus
3. Lire les fichiers ligne à ligne et conserver uniquement les lignes qui ne contiennent pas d'information manquante ("")
4. Sauvegarder les fichiers réparer dans un répertoir séparé.


In [1]:
# retrieving  all xml file from AFD
from urllib.request import urlopen
from bs4 import BeautifulSoup
import logging
# Set logging parameters
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
# create a file handler
handler = logging.FileHandler('IATI error fixe.log')
handler.setLevel(logging.INFO)
# add the handlers to the logger
logger.addHandler(handler)

# url du dashboard AFD où sont présentés les fichiers XML de l'AFD
url="http://dashboard.iatistandard.org/publisher/afd.html"

#use beautiful soup to parse it
html=urlopen(url).read()
soup=BeautifulSoup(html, "html.parser")

# keep all anchor link
tags=soup('a')

#in each anchor look for an afd url ("https://www.data.gouv.fr/s/resources/")
for tag in tags:
    # Retrieve all tag
    if "https://www.data.gouv.fr/s/resources/" in tag.get('href',None): 
        error_file=tag.get('href',None)
        logger.info("---- Process ----- ")
        logger.info("XML file to be checked = "+error_file)
        logger.info("Link to IATI validator= "+"http://validator.iatistandard.org/?url="+error_file)
      
        # Process the file
        url_error=error_file
        filename=url_error[-13:]
        #print(filename)
        page= urlopen(url_error)

        # Download original file
        with open("original/"+filename, "wb") as xml: xml.write(page.read())
        logger.info("----------")
        print("Original:"+filename+ "- Status: downloaded")
        print("-----------")
        logger.info("Original:"+filename+ "- Status: downloaded")
        logger.info("----------")
        #Open the url
        page= urlopen(url_error)
        #for each line in the page source, look for defined set of characters (link containing pdf) and then add them to a list
        with open("repaired/"+filename,'wb') as output:
            #incremental variable to store error
            n=0
            for line in page:
                # Chekc if empty string ("") are not present, if so we can write the line
                if '""' not in line.decode("utf8"): 
                    output.write(line) 
                else: 
                    n=n+1
                    logger.info("Error n°" +str(n)+ " was:"+line.decode("utf8"))
        logger.info("----------")            
        logger.info("Original:"+filename+ "- Status:"+str(n)+" Errors Repaired, file saved")
        print("Original:"+filename+ "- Status:"+str(n)+" Errors Repaired, file saved")
        print("-----------")
        logger.info("--------- ")

Original:fr-AFD-AF.xml- Status: downloaded
-----------
Original:fr-AFD-AF.xml- Status:8 Errors Repaired, file saved
-----------
Original:fr-AFD-BD.xml- Status: downloaded
-----------
Original:fr-AFD-BD.xml- Status:4 Errors Repaired, file saved
-----------
Original:fr-AFD-BF.xml- Status: downloaded
-----------
Original:fr-AFD-BF.xml- Status:22 Errors Repaired, file saved
-----------
Original:fr-AFD-BR.xml- Status: downloaded
-----------
Original:fr-AFD-BR.xml- Status:7 Errors Repaired, file saved
-----------
Original:fr-AFD-BJ.xml- Status: downloaded
-----------
Original:fr-AFD-BJ.xml- Status:24 Errors Repaired, file saved
-----------
Original:fr-AFD-BO.xml- Status: downloaded
-----------
Original:fr-AFD-BO.xml- Status:10 Errors Repaired, file saved
-----------
Original:fr-AFD-BR.xml- Status: downloaded
-----------
Original:fr-AFD-BR.xml- Status:7 Errors Repaired, file saved
-----------
Original:fr-AFD-CD.xml- Status: downloaded
-----------
Original:fr-AFD-CD.xml- Status:17 Errors Repai